In [ ]:
stops = spark.read.format('orc').load('/data/sbb/timetables/orc/stops/000000_0')

In [75]:
stops_pos = stops.join(stops_distance, 'stop_id').select(col('stop_id'), col('stop_lat'), col('stop_lon'))
stops_pos = stops_pos.select(col('stop_id').alias('stop_id_1'), col('stop_lat').alias('stop_lat_1'), col('stop_lon').alias('stop_lon_1'))
stops_pos = stops_pos.crossJoin(stops_pos.select(col('stop_id_1').alias('stop_id_2'), col('stop_lat_1').alias('stop_lat_2'), col('stop_lon_1').alias('stop_lon_2')))
stops_pos = stops_pos.where(col('stop_id_1') != col('stop_id_2'))
stops_pos.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------+---------------+-----------+----------------+----------------+
|stop_id_1|      stop_lat_1|     stop_lon_1|  stop_id_2|      stop_lat_2|      stop_lon_2|
+---------+----------------+---------------+-----------+----------------+----------------+
|  8500926|47.4236270123012|8.4031825286317|    8502186|47.3934058321612|8.39894248049007|
|  8500926|47.4236270123012|8.4031825286317|8502186:0:1|47.3934666445388|8.39894248049007|
|  8500926|47.4236270123012|8.4031825286317|8502186:0:2|47.3935274568464|8.39894248049007|
|  8500926|47.4236270123012|8.4031825286317|   8502186P|47.3934058321612|8.39894248049007|
|  8500926|47.4236270123012|8.4031825286317|    8502187|47.3646945560768|8.37709545277724|
|  8500926|47.4236270123012|8.4031825286317|8502187:0:1|47.3647554015789|8.37709545277724|
|  8500926|47.4236270123012|8.4031825286317|8502187:0:2|47.3648162470108|8.37709545277724|
|  8500926|47.4236270123012|8.4031825286317|   8502187P|47.3646945560768|8.37709545277724|

In [76]:
stops_pos_dist = stops_pos.withColumn('distance', compute_distance(col('stop_lat_1'), col('stop_lon_1'), col('stop_lat_2'), col('stop_lon_2')))
stops_pos_dist.cache()
stops_pos_dist.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------+---------------+-----------+----------------+----------------+---------+
|stop_id_1|      stop_lat_1|     stop_lon_1|  stop_id_2|      stop_lat_2|      stop_lon_2| distance|
+---------+----------------+---------------+-----------+----------------+----------------+---------+
|  8500926|47.4236270123012|8.4031825286317|    8502186|47.3934058321612|8.39894248049007|3375.1602|
|  8500926|47.4236270123012|8.4031825286317|8502186:0:1|47.3934666445388|8.39894248049007|3368.4294|
|  8500926|47.4236270123012|8.4031825286317|8502186:0:2|47.3935274568464|8.39894248049007|3361.6992|
|  8500926|47.4236270123012|8.4031825286317|   8502186P|47.3934058321612|8.39894248049007|3375.1602|
|  8500926|47.4236270123012|8.4031825286317|    8502187|47.3646945560768|8.37709545277724|6841.6157|
|  8500926|47.4236270123012|8.4031825286317|8502187:0:1|47.3647554015789|8.37709545277724| 6835.137|
|  8500926|47.4236270123012|8.4031825286317|8502187:0:2|47.3648162470108|8.37709545277724|6

In [77]:
stops_pos_dist.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3532520

In [78]:
walking_edges = stops_pos_dist.select(col('stop_id_1').alias('source'), col('stop_id_2').alias('target'), col('distance'))\
                                        .where(col('distance') <= 500)\
                                        .withColumn('duration', col('distance')/50)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
walking_edges.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+---------+-------------------+
|     source|     target| distance|           duration|
+-----------+-----------+---------+-------------------+
|    8500926|    8590616|122.61607| 2.4523214721679687|
|    8500926|    8590737| 300.6712|  6.013424072265625|
|    8502186|8502186:0:1|6.7610297|0.13522059440612794|
|    8502186|8502186:0:2|13.522052| 0.2704410362243652|
|    8502186|   8502186P|      0.0|                0.0|
|    8502186|    8502270|478.78476|  9.575695190429688|
|    8502186|8502270:0:1|485.54355|  9.710870971679688|
|    8502186|    8590200| 483.6799|  9.673598022460938|
|    8502186|    8590203|455.34915|  9.106983032226562|
|8502186:0:1|    8502186|6.7610297|0.13522059440612794|
|8502186:0:1|8502186:0:2| 6.761022| 0.1352204418182373|
|8502186:0:1|   8502186P|6.7610297|0.13522059440612794|
|8502186:0:1|    8502270| 472.0255|   9.44051025390625|
|8502186:0:1|8502270:0:1| 478.7842|  9.575684204101563|
|8502186:0:1|    8590200| 484.2558|  9.685115966

## Save to local

In [80]:
%%spark -o walking_edges -n 1000000

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
%%local
walking_edges.to_pickle('walking_times.pickle')